<a href="https://colab.research.google.com/github/eyyupcanisler/impark-internship/blob/main/impark_table_to_excel_advance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import gspread
from google.colab import auth
from google.auth import default
from gspread_dataframe import get_as_dataframe, set_with_dataframe

# Authenticate and connect to Google Sheets
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open the Google Sheet and get the last sheet (assuming it's the one we just updated)
sheet1 = gc.open_by_url('insert link').sheet1

data1 = sheet1.get_all_values()
df1 = pd.DataFrame(data1[1:], columns=data1[0])

# Load the output3.xlsx file
df3 = pd.read_excel('connect your file path')

In [ ]:
df1.head()

In [ ]:
df3.head()

In [ ]:
df3 = df3.rename(columns={'kurum': 'OKUL ADI'})

In [ ]:
df3[df3['plus_paket'].str.contains("KÖK", na=False)]

In [ ]:
sample_dict = df3["plus_paket"]

import ast


# Convert the string to a list of dictionaries
for i, item in enumerate(sample_dict):
  sample_dict[i] = ast.literal_eval(item)

In [ ]:
sample_dict.head()

In [ ]:
result = []
for i, row in enumerate(sample_dict):
  for item in row:
    if "plus_paket_0" in item and "KÖK" in item["plus_paket_0"]:
      result.append((i, item))

result

In [ ]:
# Extract indices and values, handling potential KeyError
indices = [item[0] for item in result]
values = []
for item in result:
    try:
        values.append(item[1]['plus_paket_0'])
    except KeyError:
        values.append(None)  # Append None for missing 'plus_paket_0'

# Create a Series with indices and values
series = pd.Series(values, index=indices)

# Group values by index and join them into a single cell
df = series.groupby(series.index).apply(lambda x: ', '.join(x.dropna())).to_frame(name='plus_paket_0')

# Reindex the DataFrame to include all indices from df3, filling missing values with empty strings
df = df.reindex(df3.index, fill_value='')

df.head()

In [ ]:
df3['plus_paket'] = df

df3.head()

In [ ]:
# prompt: I want to add plus_paket column to df1 and I need to make match with OKUL ADI values

# Merge df1 and df3 based on 'OKUL ADI'
merged_df = pd.merge(df1, df3[['OKUL ADI', 'plus_paket']], on='OKUL ADI', how='left')

# Fill missing values in 'plus_paket' with empty strings
merged_df['plus_paket'].fillna('', inplace=True)

# Update df1 with the merged data
df1 = merged_df.copy()

In [ ]:
# Assuming 'df' is your DataFrame
df1 = df1.rename(columns={'plus_paket': 'KÖK'})


In [ ]:
columns_to_drop = ['AÇILDI', 'ÜST YÖNETİCİ', 'DİJİREAD-TERCİH ROBOTU',
       'ÖZEL WEB-MOBİL UYGULAMA BEDELİ', 'SUB DOMAIN', 'DOMAIN']
df1 = df1.drop(columns=columns_to_drop)

In [ ]:
new_order = ['TARİH', 'KÖK', 'SATILAN ÜRÜN ADI', 'SİPARİŞ KONTENJAN ADEDİ',
       'İŞLER MUHASEBE LOGO - SİPARİŞ NUMARASI', 'MUHASEBE AÇIKLAMA',
       'SİPARİŞ ONAYLAYAN KİŞİ', 'İL', 'İLÇE',
       'İŞLER BAYİ-FRANCHİSE KURUMSAL ADI', 'OKUL TÜRÜ (DEVLET - ÖZEL - KURS)',
       'OKUL ADI', 'OKUL İRTİBAT KİŞİ', 'OKUL İRTİBAT TELEFONU ', 'EMAIL',
       'SORUMLU VEYA GÖRÜŞEN PERSONEL', 'EĞİTİM VEREN PERSONEL',
       'AÇIKLAMALAR',
       'KAYIT İŞLEMLERİ ANLATILDI', 'Kaynak dağılımı yapıldı.',
       'iŞLEM TAMAMLANDI', 'CEVAPSIZ ÇAĞRI SAYISI','SİSTEM TARİHİ']

df1 = df1[new_order]

In [ ]:
df1.head()

In [ ]:
# Filter the DataFrame to include rows where the 'KÖK' column is not empty
df_kok = df1[df1['KÖK'] != '']

df_kok.head()

In [ ]:
# Open the specific sheet "djdm-satıslar-2024-2025-ölçme"
sheet_kok = gc.open_by_url('insert_your_final_sheet').worksheet('worksheet_name')


In [ ]:
# Clear existing data in the new sheet (optional)
sheet_kok.clear()

In [ ]:
# Write the filtered DataFrame to the new sheet
set_with_dataframe(sheet_kok, df_kok)